In [101]:
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
from math import sqrt
import warnings

In [6]:
key1 = '6LgngVxBC_124fd8J9otcFIuQjOuZf4ruooOHRfpZXeLjxNQDeTq3HT1xbpzYlEXy2-qPos4qxQ4eoJPfiLdbFP15mRETfG-RJlT8egtQ5J3Efsa0udq8SrekEvuZXYx'
key2 = 'oG72h4lS--zvB94kJVRJV1UvB_U5YqoecbcBw-6juM-3SbuXwaEdOuzToX5QHbrlmY09nLtOFCBCZ2-CcjNREGoymN1Ja_TtcP5lbpf8OZyEgG6gHcSyQwhbTfrvZXYx'
key3 = 'to_0IwmkQRcu_g1Lz8UkHiPp_IjT9BVX4mFE1wAOuWtNaK41_hAeUgYMknwxyhPE8x52DAhNJKe-DaaK3dKeeebNnJhByuhjTtqA_hFRTSU2NovZyeTcXffv2f3vZXYx'

# Get Basic Information

In [8]:
api_search = 'https://api.yelp.com/v3/businesses/search'

params_search = {
    "location": "Sacramento, CA",
    "term":"Chinese",
    "limit":50,
    "offset":0
}

headers = {
    'Authorization': f'Bearer {key1}'
}

all_results = []
while True:
    time.sleep(0.2)
    response = requests.get(api_search, params=params_search, headers=headers)
    if response.status_code == 200:
        data = response.json()
        businesses = data.get("businesses", [])
        all_results.extend(businesses)
    
        if len(businesses) < 49:  # 如果返回的结果少于请求的限制，则说明已经获取了所有结果
            break
    
        params_search["offset"] += 50  # 增加offset以获取下一页的结果
        print(params_search["offset"])
        
    elif response.status_code == 413:
        print("You have either exceeded your daily quota.")

50
100
150
200
250
300
350
400
450


In [620]:
for i in range(len(all_results)):
# for i in range(5):
    all_results.loc[i, 'latitude'] = all_results['coordinates'][i]['latitude']
    all_results.loc[i, 'longitude'] = all_results['coordinates'][i]['longitude']
    address2 = all_results['location'][i]['address2']
    if address2:
        all_results.loc[i, 'address'] = all_results['location'][i]['address1'] + " " + all_results['location'][i]['address2'] + " " + "Sacramento"
    else:
        all_results.loc[i, 'address'] = all_results['location'][i]['address1'] + " " + "Sacramento"
        
    cate = []
    n_type = len(all_results['categories'][i])
    for j in range(n_type):
        cate.append(all_results['categories'][i][j]['title'])
#     print(cate)
    all_results['category'][i] = cate
    warnings.filterwarnings('ignore')

In [298]:
all_results.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance,latitude,longitude,address,category
0,uTqmKshAoHYHISPE7n6bNQ,fortune-chinese-food-sacramento-2,Fortune Chinese Food,https://s3-media3.fl.yelpcdn.com/bphoto/7hq15-...,False,https://www.yelp.com/biz/fortune-chinese-food-...,455,"[{'alias': 'chinese', 'title': 'Chinese'}]",4.4,"{'latitude': 38.5414479054582, 'longitude': -1...","[delivery, pickup]",$$,"{'address1': '2805 12th Ave', 'address2': 'Ste...",+19164526628,(916) 452-6628,4455.184436,38.541448,-121.474960,2805 12th Ave Ste 4 Sacramento,[Chinese]
1,-DjZjrHcelHAvMKkUHNALA,jade-garden-sacramento-2,Jade Garden,https://s3-media1.fl.yelpcdn.com/bphoto/4t_4ws...,False,https://www.yelp.com/biz/jade-garden-sacrament...,97,"[{'alias': 'chinese', 'title': 'Chinese'}]",4.3,"{'latitude': 38.54817391261898, 'longitude': -...",[],$$,"{'address1': '9185 Kiefer Blvd', 'address2': N...",+19163621771,(916) 362-1771,7839.281708,38.548174,-121.358335,9185 Kiefer Blvd Sacramento,[Chinese]
2,awbNonET06fRkvg6ERysTA,journey-to-the-dumpling-midtown-sacramento,Journey to the Dumpling - Midtown,https://s3-media3.fl.yelpcdn.com/bphoto/tBLIQA...,False,https://www.yelp.com/biz/journey-to-the-dumpli...,265,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",4.3,"{'latitude': 38.56845, 'longitude': -121.48219}",[restaurant_reservation],$$,"{'address1': '1700 21st St', 'address2': '', '...",+19168224473,(916) 822-4473,3456.852387,38.568450,-121.482190,1700 21st St Sacramento,"[Chinese, Noodles, Soup]"
3,uSh45fmNiQz7x4l-zF0ixA,t-and-z-chinese-restaurant-sacramento,T & Z Chinese Restaurant,https://s3-media4.fl.yelpcdn.com/bphoto/jKt5a-...,False,https://www.yelp.com/biz/t-and-z-chinese-resta...,551,"[{'alias': 'chinese', 'title': 'Chinese'}]",4.4,"{'latitude': 38.6129471144324, 'longitude': -1...","[delivery, pickup]",$$,"{'address1': '2550 W El Camino Ave', 'address2...",+19166491514,(916) 649-1514,8019.202014,38.612947,-121.519422,2550 W El Camino Ave Ste 8 Sacramento,[Chinese]
4,tjBXGjWKllYhy8DHPhI-0Q,yue-huang-sacramento-2,Yue Huang,https://s3-media2.fl.yelpcdn.com/bphoto/xsiH7T...,False,https://www.yelp.com/biz/yue-huang-sacramento-...,1174,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",3.6,"{'latitude': 38.6384473758722, 'longitude': -1...","[delivery, pickup]",$$,"{'address1': '3860 Truxel Rd', 'address2': '',...",+19166213737,(916) 621-3737,9030.219261,38.638447,-121.503844,3860 Truxel Rd Sacramento,"[Seafood, Cantonese, Dim Sum]"


In [523]:
all_categories = []
for i in all_results['categories']:
    n_type = len(i)
    for j in range(n_type):
        all_categories.append(i[j]['title'])
all_categories = set(all_categories)
categories = []
for i in all_categories:
    categories.append(i)
# categories

In [607]:
all_info = all_results[['name', 'id', 'review_count', 'rating', 'latitude', 'longitude', 'address','category']]
all_info

,name,id,review_count,rating,latitude,longitude,address,category
0,Fortune Chinese Food,uTqmKshAoHYHISPE7n6bNQ,455,4.4,38.541448,-121.474960,2805 12th Ave Ste 4 Sacramento,[Chinese]
1,Jade Garden,-DjZjrHcelHAvMKkUHNALA,97,4.3,38.548174,-121.358335,9185 Kiefer Blvd Sacramento,[Chinese]
2,Journey to the Dumpling - Midtown,awbNonET06fRkvg6ERysTA,265,4.3,38.568450,-121.482190,1700 21st St Sacramento,"[Chinese, Noodles, Soup]"
3,T & Z Chinese Restaurant,uSh45fmNiQz7x4l-zF0ixA,551,4.4,38.612947,-121.519422,2550 W El Camino Ave Ste 8 Sacramento,[Chinese]
4,Yue Huang,tjBXGjWKllYhy8DHPhI-0Q,1174,3.6,38.638447,-121.503844,3860 Truxel Rd Sacramento,"[Seafood, Cantonese, Dim Sum]"
...,...,...,...,...,...,...,...,...
491,Olive Garden Italian Restaurant,cMACJTd-jI596Fn9xixkxQ,486,2.9,38.666110,-121.272606,5445 Sunrise Blvd Sacramento,"[Italian, Salad, Soup]"
492,Preethi Indian Cuisine,ENmi2GlUjrm8A5nKTv-czA,372,3.7,38.543662,-121.739428,715 2nd St Sacramento,"[Indian, Vegan, Soup]"
493,Olive Garden Italian Restaurant,5iXkCdDM1IlWWTVWSwdO3w,498,3.4,38.770871,-121.265159,1160 Galleria Blvd Sacramento,"[Italian, Salad, Soup]"
494,Panera Bread,lQErN-2_H0PcNJIfOxCtlA,404,3.1,38.424826,-121.387587,8519 Bond Rd Ste 106 Sacramento,"[Sandwiches, Salad, Soup]"


# open time

In [354]:
api_buss = 'https://api.yelp.com/v3/businesses/' + buss_id
key4 = 'j8Q2sqYK3mLSJxATCqSUyvMUxfRHURDoIMdRJ6V-G8WohBcs9hEsXa1kHeLbOyZCwvFRGvOVlSUdQLHQVUu1OMKKFTAS6sRTQh6syLSP0UHSwxj4yNrduxWkOFPuZXYx'
headers = {
    'Authorization': f'Bearer {key4}'
}

In [356]:
open_time = []
for buss_id in all_info['id']:
    api_buss = 'https://api.yelp.com/v3/businesses/' + buss_id
    response = requests.get(api_buss, headers=headers)
    time.sleep(1)
    if response.status_code == 200:
        data = response.json()
        name = data['name']
#         review_count = data['review_count']
#         rating = data['rating']
#         address2 = data['location']['address2']
#         if address2:
#             address = data['location']['address1'] + " " + data['location']['address2'] + " " + data['location']['city']
#         else:
#             address = data['location']['address1'] + " " + data['location']['city']
        open_day = []
        hours = re.findall(r'hours', response.text)
        if hours:
            for i in data['hours'][0]['open']:
#             is_overnight = i['is_overnight']
#             start = i['start']
#             end = i['end']
                day = i['day']
                open_day.append(day)
        else:
            open_day.append('Na')
#             print(buss_id)
        open_time.append(list(set(open_day)))

    else:
        open_time.append(['Na'])
        print(buss_id)
        print(response.status_code)

gBvWiu-2aW7nJyM7F1ayYg
403
hZsVSt1RqN-DH1COT9ZEEg
403
0j2d8A-W5F0DzUMmSXQjLQ
403
M6l6Csedbr4Q-VHQ0FayCg
403
ovRKM_ah5Q--Ur44Bb-pnw
403
e6iug3vEpAGdGt97Gs1yZQ
403
dDFRLX-52hcL0hwHCh6TRQ
403
Ht_UU6rHlakRLxe-1JiR_g
403
vj8iMqK745cMFCDDgVmkbA
403
nzrdJuskhpCnbRfIK_rceg
403
sFd7MROvLxkzXdtlP04YoA
403
k6Y3A43S_Q76nXCTcsXPIw
403
DvgVuTuRdrui1n1I-e4ClQ
403


In [608]:
open_time1 = []
for i in open_time:
    open_time1.append({'open_time':i})
open_time1 = pd.DataFrame(open_time1)

all_info = pd.concat([all_info, open_time1], axis=1)
# all_info.rename(columns={'name':'Name', 'id':'ID', 'review_count': 'Reviewed', 'rating':'Rating', 'latitude':'Latitude', 'longitude':'Longitude', 'address':'Address', 'category':'Categories', 'open_time':'Open_days'}, inplace=True)
all_info.rename(columns={'latitude':'lat', 'longitude':'lon'}, inplace=True)
all_info[0:10]

,name,id,review_count,rating,lat,lon,address,category,open_time
0,Fortune Chinese Food,uTqmKshAoHYHISPE7n6bNQ,455,4.4,38.541448,-121.474960,2805 12th Ave Ste 4 Sacramento,[Chinese],"[1, 2, 3, 4, 5, 6]"
1,Jade Garden,-DjZjrHcelHAvMKkUHNALA,97,4.3,38.548174,-121.358335,9185 Kiefer Blvd Sacramento,[Chinese],"[1, 2, 3, 4, 5, 6]"
2,Journey to the Dumpling - Midtown,awbNonET06fRkvg6ERysTA,265,4.3,38.568450,-121.482190,1700 21st St Sacramento,"[Chinese, Noodles, Soup]","[0, 1, 2, 3, 4, 5, 6]"
3,T & Z Chinese Restaurant,uSh45fmNiQz7x4l-zF0ixA,551,4.4,38.612947,-121.519422,2550 W El Camino Ave Ste 8 Sacramento,[Chinese],"[0, 2, 3, 4, 5, 6]"
4,Yue Huang,tjBXGjWKllYhy8DHPhI-0Q,1174,3.6,38.638447,-121.503844,3860 Truxel Rd Sacramento,"[Seafood, Cantonese, Dim Sum]","[0, 1, 2, 3, 4, 5, 6]"
5,Hidden Dumpling House Midtown,8lq7XpBXsIbSFEi5TLzAjg,325,4.0,38.567359,-121.471809,2801 P St Sacramento,"[Chinese, Noodles, Bubble Tea]","[0, 1, 2, 3, 4, 5, 6]"
6,Chef Bo,AmMdJ61S2yljki10d5EsTA,471,3.8,38.573623,-121.410413,2310 Fair Oaks Blvd Ste A Sacramento,[Chinese],"[0, 1, 2, 3, 4, 5]"
7,Old Gui Lin,EvKjZ-0uMNai9fnkOFCqfg,61,4.3,38.527290,-121.496010,5131 Freeport Blvd Sacramento,"[Chinese, Noodles]","[0, 1, 2, 3, 4, 5, 6]"
8,Frank Fat's,gYogsRwJcr8onx7-dzAjvQ,1024,4.1,38.578400,-121.496840,806 L St Sacramento,[Chinese],"[1, 2, 3, 4, 5, 6]"
9,The Flame Cafe,DI4X4NijlmrbA_i80QHsbw,477,4.5,38.494761,-121.456720,4220 Florin Rd Ste 107 Sacramento,"[Hong Kong Style Cafe, Chinese]","[0, 1, 3, 4, 5, 6]"


# Get address and Rate

In [10]:
# address_rate = []

# for i in range(len(all_results)):
#     info = pd.DataFrame(all_results['location'][i])
#     address1 = info['address1'][0]
#     rating = all_results['rating'][0]
#     if address1:
#         address_rate.append({"address":address1+" "+"Sacramento"})

In [524]:
# address_rate = pd.DataFrame(address_rate)
# address_rate

## Get Latitude and Lontitude Information from Address

### nominatim

In [525]:
# url_location = 'https://nominatim.openstreetmap.org/search'
# response = requests.get(url_location)
# response.status_code

In [526]:
# params = {
#         "q": address_rate['address'][1]+',CA',
#         "format":"json",
#         "limit":1
#         }
# response = requests.get(url_location, params)
# response.status_code

In [527]:
# coordination = []
# for i in range(len(address_rate[0:10])):
#     time.sleep(0.2)
#     params = {
#         "q": address_rate['address'][i]+',CA',
#         "format":"json",
#         "limit":1
#         }
#     response = requests.get(url_location, params)
#     if response.status_code == 200:
#         result = response.json()
#         if result:
#             lat = float(result[0].get("lat", []))
#             lon = float(result[0].get("lon", []))
#             coordination.append({"name": all_results['name'][i], "rate":all_results['rating'][i], "lat":lat, "lon":lon, "review_count":all_results['review_count'][i]})

### opencage

In [105]:
# api_key = '75bf8535ebdb4cc8a7b48a21ade3b0f5'
# url_location = 'https://api.opencagedata.com/geocode/v1/json'

In [51]:
# params = {
#     'q': '9185 Kiefer Blvd Sacramento',  
#     'key': api_key,  # API密钥
#     'limit': 1  # 返回结果的最大数量
# }
# response = requests.get(url, params=params)

In [52]:
# def dms_to_dd(dms):
#     degrees, minutes, seconds, direction = re.split('[°\'"]+', dms)
#     dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60)
#     if direction in ('S', 'W'):
#         dd *= -1
#     return dd

In [112]:
# coordination = []
# for i in range(len(address_rate)):
#     time.sleep(0.1)
#     params = {
#         'q': address_rate['address'][i]+',CA', 
#         'key': api_key,
#         'limit': 1
#     }
#     response = requests.get(url_location, params)
#     if response.status_code == 200:
#         result = response.json()
#         if result:
#             lat = result['results'][0]['annotations']['DMS']['lat']
#             lon = result['results'][0]['annotations']['DMS']['lng']
#             lat_dd = dms_to_dd(lat)
#             lon_dd = dms_to_dd(lon)
#             coordination.append({"name": all_results['name'][i], "rate":all_results['rating'][i], "lat":lat_dd, "lon":-lon_dd, "review_count":all_results['review_count'][i]})
# #             print(i)
#     else:
#         print("request faild", response.status_code)

In [528]:
# coordination = pd.DataFrame(coordination)
# coordination

In [491]:
all_info

,name,id,review_count,rating,lat,lon,address,category,open_time
0,Fortune Chinese Food,uTqmKshAoHYHISPE7n6bNQ,455,4.4,38.541448,-121.474960,2805 12th Ave Ste 4 Sacramento,[Chinese],"[1, 2, 3, 4, 5, 6]"
1,Jade Garden,-DjZjrHcelHAvMKkUHNALA,97,4.3,38.548174,-121.358335,9185 Kiefer Blvd Sacramento,[Chinese],"[1, 2, 3, 4, 5, 6]"
2,Journey to the Dumpling - Midtown,awbNonET06fRkvg6ERysTA,265,4.3,38.568450,-121.482190,1700 21st St Sacramento,"[Chinese, Noodles, Soup]","[0, 1, 2, 3, 4, 5, 6]"
3,T & Z Chinese Restaurant,uSh45fmNiQz7x4l-zF0ixA,551,4.4,38.612947,-121.519422,2550 W El Camino Ave Ste 8 Sacramento,[Chinese],"[0, 2, 3, 4, 5, 6]"
4,Yue Huang,tjBXGjWKllYhy8DHPhI-0Q,1174,3.6,38.638447,-121.503844,3860 Truxel Rd Sacramento,"[Seafood, Cantonese, Dim Sum]","[0, 1, 2, 3, 4, 5, 6]"
...,...,...,...,...,...,...,...,...,...
491,Olive Garden Italian Restaurant,cMACJTd-jI596Fn9xixkxQ,486,2.9,38.666110,-121.272606,5445 Sunrise Blvd Sacramento,"[Italian, Salad, Soup]","[0, 1, 2, 3, 4, 5, 6]"
492,Preethi Indian Cuisine,ENmi2GlUjrm8A5nKTv-czA,372,3.7,38.543662,-121.739428,715 2nd St Sacramento,"[Indian, Vegan, Soup]","[0, 1, 2, 3, 4, 5, 6]"
493,Olive Garden Italian Restaurant,5iXkCdDM1IlWWTVWSwdO3w,498,3.4,38.770871,-121.265159,1160 Galleria Blvd Sacramento,"[Italian, Salad, Soup]","[0, 1, 2, 3, 4, 5, 6]"
494,Panera Bread,lQErN-2_H0PcNJIfOxCtlA,404,3.1,38.424826,-121.387587,8519 Bond Rd Ste 106 Sacramento,"[Sandwiches, Salad, Soup]","[0, 1, 2, 3, 4, 5, 6]"


In [609]:
all_info1 = all_info[abs(all_info['lat']-all_info['lat'].mean())>1*sqrt(all_info['lat'].var())]
all_info2 = all_info[abs(all_info['lat']-all_info['lat'].mean())<1*sqrt(all_info['lat'].var())]

## Add review count into threshold

In [611]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371
    return c * r

def adjust_zoom(df_filtered):
    max_distance = 0
    for i in range(len(df_filtered)):
        for j in range(i+1, len(df_filtered)):
            distance = haversine(df_filtered.iloc[i]['lon'], df_filtered.iloc[i]['lat'],
                                 df_filtered.iloc[j]['lon'], df_filtered.iloc[j]['lat'])
            max_distance = max(max_distance, distance)
    if max_distance == 0:
        return 12
    return max(4, round(14 - log(max_distance + 1)))

In [616]:
def generate_adjusted_map(threshold, threshold2, category_filter, day_filter, df):
    df_filtered = df[(df['rating'] >= threshold) & (df['review_count'] >= threshold2)]
    if category_filter != 'All':
        df_filtered = df_filtered[df_filtered['category'].apply(lambda x: category_filter in x)]
    
    if day_filter != 'All':
        df_filtered = df_filtered[df_filtered['open_time'].apply(lambda x: day_filter in x)]
    
    center_lat, center_lon = df['lat'].mean(), df['lon'].mean()  # Default center
    zoom_level = 12
    
    if not df_filtered.empty:
        center_lat, center_lon = df_filtered['lat'].mean(), df_filtered['lon'].mean()
        zoom_level = adjust_zoom(df_filtered)
    
    m = folium.Map(location=[center_lat, center_lon], zoom_start=zoom_level)
    
    if df_filtered.empty:
        return m

    for _, row in df_filtered.iterrows():
        icon_size = max(5, (row['rating']*50+row['review_count']) / 100)
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=icon_size,
            popup=f"{row['name']}, Rating: {row['rating']}, Reviews: {row['review_count']}",
            color='red',
            fill=True,
        ).add_to(m)
    
    return m

days = [0, 1, 2, 3, 4, 5, 6]
days_dropdown = widgets.Dropdown(options=['All'] + days, value='All', description='Day:')

@interact
def display_map(Rating=widgets.FloatSlider(min=all_info2['rating'].min(), max=all_info2['rating'].max(), step=0.1, value=4.0),
                Review=widgets.IntSlider(min=all_info2['review_count'].min(), max=all_info2['review_count'].max(), step=1, value=100),
                Category=category_dropdown,
                Day=days_dropdown):
    return generate_adjusted_map(Rating, Review, Category, Day, all_info2)

interactive(children=(FloatSlider(value=4.0, description='Rating', max=5.0), IntSlider(value=100, description=…

In [612]:
@interact
def display_map(Rating=widgets.FloatSlider(min=all_info1['rating'].min(), max=all_info1['rating'].max(), step=0.1, value=4.0),
                Review=widgets.IntSlider(min=all_info1['review_count'].min(), max=all_info1['review_count'].max(), step=1, value=100),
                Category=category_dropdown,
                Day=days_dropdown):
    return generate_adjusted_map(Rating, Review, Category, Day, all_info1)

interactive(children=(FloatSlider(value=1.9, description='Rating', max=1.9, min=1.9), IntSlider(value=78, desc…

In [613]:
len(all_info2)

495